In [2]:
import torch
import torch.nn as nn
from sklearn.feature_extraction import image
from sklearn.datasets import load_sample_image
import numpy as np

# 0. Imaage 

In [10]:
img = load_sample_image('china.jpg')

In [12]:
img.shape

(427, 640, 3)

In [13]:
patches = image.extract_patches_2d(img, (16, 16))

In [15]:
patches.shape

(257500, 16, 16, 3)

In [17]:
patches[0].shape

(16, 16, 3)

In [18]:
patch = patches[0]